<h1 align=center><font size = 5><h1 align=center><font size = 5>Capstone Project - The Neighborhoods of Paris - Restaurants</font></h1></font></h1>

# Introduction/Business Problem section

## 1.1 Description of the problem 

Paris is the capital and the most popular city in France. I enjoyed my winter break two years ago in paris where the delicious food there is very impressive! For someone who would like to start the new business in paris, what can we provide the support information and where can we recommend that they open it ?

## 1.2 Discussion of the Background 

Paris is currently ranked 3rd in the global economic power index. The City of Paris is the centre and seat of government of the Paris Region, which has an estimated official 2020 population of 12,278,210, or about 18 percent of the population of France. 

Paris has been famous for its restaurants. Also, Paris received 38 million visitors in 2019, measured by hotel stays, with the largest numbers of foreign visitors coming from the United States, the United Kingdom, Germany, and China. It was ranked as the second most visited travel destination in the world in 2019, after Bangkok and just ahead of London. These might provide great opportunities for restaurant to offer decilious food for travellers and workers. 

The city of Paris is divided into 20 arrondissements such as Louvre, Bourse, Temple, Hôtel-de-Ville, Panthéon etc. It is difficult to figure out where will be the great place to open the restaurant. In this project, I will outline the data preparation and then use Foursquare location data and clustering methods to group the Arriondissements to different group by their restaurant venues information. 

## 1.3 Target the Audience 

What kind of people will be interested in the project ?

1. Businees people who would like to invest or open a restaurant 
2. Workers or students who prefer to find the close restaurants enjoy the lunch/dinner time. 
3. Data Scientists who want to implement different analysis based on the datasets and information provided.

# Data section 

## 2.1 Web Scrapping from Wikipedia

Scrapping the following Wikipedia page, https://en.wikipedia.org/wiki/Arrondissements_of_Paris. I’ve used requests and Beautifulsoup4 library to create a data-frame containing name of the 20 Arrondissements, Name, Area, Population, Density, Peak of Population, Mayor. We start as below and simply the dataframe to only focus on the Arrondissements, Name, Area and Population. 

In [18]:
from bs4 import BeautifulSoup 
import requests # library to handle requests

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium

url = "https://en.wikipedia.org/wiki/Arrondissements_of_Paris"
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

# find the table 

My_table = soup.find('table', class_='wikitable')
table_A = []
for tr in My_table.findAll('tr'):
    td = tr.find_all('td')
    if len(td)==7:
        row = [i.text for i in td]
        table_A.append(row)

# define the dataframe columns
column_names = ['Arrondissements', 'Name', 'Area','Population','Density','Peak of Population','Mayor']

# instantiate the dataframe
df = pd.DataFrame(table_A, columns=column_names)

# simplify the data
df_new=df.drop(['Density','Peak of Population','Mayor'], axis=1)


#restore the data for length!=7 into a new dataframe
dict = {'Arrondissements':['2nd (IIe) RAdministratively part of Paris Centre', '3rd (IIIe) RAdministratively part of Paris Centre', '4th (IVe) RAdministratively part of Paris Centre'], 
        'Name':['Bourse', 'Temple', 'Hôtel-de-Ville'], 
        'Area':['5.59 km2 (2.16 sq mi)','5.59 km2 (2.16 sq mi)','5.59 km2 (2.16 sq mi)'],
        'Population':['100,196','100,196','100,196'],
       } 
df2 = pd.DataFrame(dict)

# insert the dataframe between the 1st line and the 2nd line, becoming the new dataframe
df3=pd.concat([df_new.iloc[:1],df2, df_new.iloc[1:]]).reset_index(drop = True)
df3

,Arrondissements,Name,Area,Population
0,1st(Ier) RAdministratively part of Paris Centre,Louvre,5.59 km2 (2.16 sq mi),"100,196"
1,2nd (IIe) RAdministratively part of Paris Centre,Bourse,5.59 km2 (2.16 sq mi),"100,196"
2,3rd (IIIe) RAdministratively part of Paris Centre,Temple,5.59 km2 (2.16 sq mi),"100,196"
3,4th (IVe) RAdministratively part of Paris Centre,Hôtel-de-Ville,5.59 km2 (2.16 sq mi),"100,196"
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"59,631"
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"41,976"
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"52,193"
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"37,368"
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"60,071"
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"90,836"


## 2.2 Get the coordinates of the Arrondissements

Obtain coordinates using Geopy library

In [9]:
geolocator = Nominatim(user_agent='myGeocoder')
df3['Coor'] = df3['Name'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df3[['Latitude','Longitude']] = df3['Coor'].apply(pd.Series)
df3.drop(['Coor'], axis=1, inplace=True)
df3

,Arrondissements,Name,Area,Population,Latitude,Longitude
0,1st(Ier) RAdministratively part of Paris Centre,Louvre,5.59 km2 (2.16 sq mi),"100,196",48.861147,2.338028
1,2nd (IIe) RAdministratively part of Paris Centre,Bourse,5.59 km2 (2.16 sq mi),"100,196",48.868630,2.341474
2,3rd (IIIe) RAdministratively part of Paris Centre,Temple,5.59 km2 (2.16 sq mi),"100,196",31.098207,-97.342785
3,4th (IVe) RAdministratively part of Paris Centre,Hôtel-de-Ville,5.59 km2 (2.16 sq mi),"100,196",43.949283,4.805260
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"59,631",41.898616,12.476833
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"41,976",49.815868,6.129675
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"52,193",48.861596,2.317909
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"37,368",4.753042,-54.044953
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"60,071",48.876019,2.339962
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"90,836",48.876126,2.359839


It seems the coordinate the Arrondissements' returns wrong since the Name of the Arrondissement in the data frame might be diffrent from the actual one. We double checked the geographical coordinate of paris.  

In [10]:
address = 'Paris'

geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


Since the geograpical coordinate of Paris are 48.8611473, 2.33802768704666. Latitude and Longtitude seems wrong for those Arrondissements far away from 48.8566969, 2.3514616.

In [11]:
# create the column of new name by adding prefix Paris and find the corresponding latitude and longitude by the new name to get the correct coordinates
geolocator = Nominatim(user_agent='myGeocoder')
df3['New Name'] = 'Paris ' + df3['Name'].astype(str)
df3['Coor'] = df3['New Name'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df3[['Latitude','Longitude']] = df3['Coor'].apply(pd.Series)

# drop the Coor and New Name to make the dataframe clean
df3.drop(['Coor'], axis=1, inplace=True)
df3.drop(['New Name'], axis=1, inplace=True)
df3

,Arrondissements,Name,Area,Population,Latitude,Longitude
0,1st(Ier) RAdministratively part of Paris Centre,Louvre,5.59 km2 (2.16 sq mi),"100,196",48.864718,2.343805
1,2nd (IIe) RAdministratively part of Paris Centre,Bourse,5.59 km2 (2.16 sq mi),"100,196",48.869756,2.341488
2,3rd (IIIe) RAdministratively part of Paris Centre,Temple,5.59 km2 (2.16 sq mi),"100,196",48.865613,2.360905
3,4th (IVe) RAdministratively part of Paris Centre,Hôtel-de-Ville,5.59 km2 (2.16 sq mi),"100,196",48.856821,2.352301
4,5th (Ve) L,Panthéon,2.541 km2 (0.981 sq mi),"59,631",48.846191,2.346079
5,6th (VIe) L,Luxembourg,2.154 km2 (0.832 sq mi),"41,976",48.850433,2.332951
6,7th (VIIe) L,Palais-Bourbon,4.088 km2 (1.578 sq mi),"52,193",48.860920,2.318035
7,8th (VIIIe) R,Élysée,3.881 km2 (1.498 sq mi),"37,368",48.846644,2.369830
8,9th (IXe) R,Opéra,2.179 km2 (0.841 sq mi),"60,071",48.872138,2.330630
9,10th (Xe) R,Entrepôt,2.892 km2 (1.117 sq mi),"90,836",48.876126,2.359839


## 2.3 create the map of Paris of 20 Arrondissements showed below:

I used python folium library to visualize geographic details of Paris and its 20 Arrondissements based on Latitude and Longtitude

In [12]:
# create map of Paris using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

## 2.4 Using FourSquare Location Data

Utilizing the Foursquare API to explore the Arrondissements and segment them.

#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = '4DBS44JI1BC3QVNDIE1UZPFDFE2OSZJZKJOUJKQUGQKNPQ5M' # your Foursquare ID
CLIENT_SECRET = 'XCF4J5WZ0TFL0OYP1C2RIWLHE4YAPK1WFGAKSPQR01UBGZNO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Now, let's get the venues that are in each 20 Arriondissements within a radius of 500 meters.

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Check the size of dataframe and its related venues 

In [17]:
paris_venues = getNearbyVenues(names=df3['Name'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )
print(paris_venues.shape)
paris_venues

Louvre
Bourse
Temple
Hôtel-de-Ville
Panthéon
Luxembourg
Palais-Bourbon
Élysée
Opéra
Entrepôt
Popincourt
Reuilly
Gobelins
Observatoire
Vaugirard
Passy
Batignolles-Monceau
Butte-Montmartre
Buttes-Chaumont
Ménilmontant
(1357, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Louvre,48.864718,2.343805,Le Louis,48.864331,2.341332,French Restaurant
1,Louvre,48.864718,2.343805,Filakia,48.865646,2.345619,Souvlaki Shop
2,Louvre,48.864718,2.343805,Le Comptoir de la Gastronomie,48.864555,2.345259,French Restaurant
3,Louvre,48.864718,2.343805,Place des Victoires,48.865759,2.341219,Plaza
4,Louvre,48.864718,2.343805,Spa Nuxe,48.864017,2.346650,Spa
...,...,...,...,...,...,...,...
1352,Ménilmontant,48.867716,2.383701,Bøti,48.864144,2.386524,Restaurant
1353,Ménilmontant,48.867716,2.383701,La Mercerie,48.865746,2.378127,Bar
1354,Ménilmontant,48.867716,2.383701,Triplettes,48.870492,2.378754,Bar
1355,Ménilmontant,48.867716,2.383701,Hôtel Les Chansonniers,48.865452,2.384100,Hotel
